In [147]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio import SearchIO
import csv
import sqlite3
import time
from multiprocessing import Pool, cpu_count
import sys
from calc_icity_ import *
import ast
from collections import defaultdict
import numpy as np

In [2]:
infile = "tnpB_targetgenes_pfam.csv"

In [7]:
pfam_df = pd.read_csv(infile).iloc[:,1:]
pfam_df.head()

,query_id,query_accession,query_len,hit_id,hit_evalue,target_length,baitp100s,icity,numer,denom,is_tnpB
0,2-oxoacid_dh,PF00198.25,233.0,241190defd57097325,1.300000e-70,392.0,"['6eb5f51e57f7125ea4', 'a316eb2bc5ed5056d1', '...",0.928571,26,28,0
1,E3_binding,PF02817.19,36.0,241190defd57097325,7.700000e-28,392.0,"['6eb5f51e57f7125ea4', 'a316eb2bc5ed5056d1', '...",0.928571,26,28,0
2,3Beta_HSD,PF01073.21,280.0,efc7276dd1d34ac243,5.100000e-15,338.0,"['c000538903999be92a', '8d91b991f320796391', '...",0.909091,20,22,0
3,Epimerase,PF01370.23,241.0,efc7276dd1d34ac243,4.500000e-61,338.0,"['c000538903999be92a', '8d91b991f320796391', '...",0.909091,20,22,0
4,GDP_Man_Dehyd,PF16363.7,332.0,efc7276dd1d34ac243,5.300000e-87,338.0,"['c000538903999be92a', '8d91b991f320796391', '...",0.909091,20,22,0


In [90]:
all_query_ids = [pfam for pfam in list(pfam_df['query_id'].sort_values().unique()) if type(pfam) == str]
all_query_ids

['2-oxoacid_dh',
 '3Beta_HSD',
 '4HBT',
 '4HBT_2',
 '4HBT_3',
 '5_nucleotid_C',
 '7TMR-DISM_7TM',
 'AAA',
 'AAA_14',
 'AAA_16',
 'AAA_18',
 'AAA_19',
 'AAA_21',
 'AAA_22',
 'AAA_25',
 'AAA_29',
 'AAA_30',
 'AAA_33',
 'AA_permease',
 'AA_permease_2',
 'ABC2_membrane',
 'ABC2_membrane_2',
 'ABC2_membrane_3',
 'ABC2_membrane_4',
 'ABC_tran',
 'ABC_trans_CmpB',
 'ACR_tran',
 'ACT',
 'ACT_4',
 'ACT_5',
 'ACT_7',
 'ACT_8',
 'ADH_N',
 'ADH_N_2',
 'ADH_zinc_N',
 'ADH_zinc_N_2',
 'AHSA1',
 'AI-2E_transport',
 'ALS_ss_C',
 'AMP-binding',
 'AMP-binding_C',
 'ANAPC4_WD40',
 'APH',
 'ASCH',
 'ASH',
 'ATP-grasp',
 'ATP-grasp_3',
 'ATPase',
 'ATPase_2',
 'Abhydrolase_1',
 'Abhydrolase_4',
 'Abhydrolase_6',
 'AbiEi_4',
 'AcaB',
 'Acetyltransf_1',
 'Acetyltransf_10',
 'Acetyltransf_3',
 'Acetyltransf_4',
 'Acetyltransf_7',
 'Acetyltransf_8',
 'Acetyltransf_9',
 'Acetyltransf_CG',
 'Acyl-ACP_TE',
 'Acyl-CoA_dh_1',
 'Acyl-CoA_dh_2',
 'Acyl-CoA_dh_M',
 'Acyl-CoA_dh_N',
 'Acyltransferase',
 'AhpC-TSA',
 'A

### look into each target gene, or hit

In [121]:
def get_hits_with_pfam_df(query_ids_suffixes, df):
    all_query_ids = [pfam for pfam in list(pfam_df['query_id'].sort_values().unique()) if type(pfam) == str]
    query_ids = []
    for query_ids_suffix in query_ids_suffixes:
        query_ids += [pfam for pfam in all_query_ids if query_ids_suffix in pfam ]    
    hits_s = df[df["query_id"].isin(query_ids)]['hit_id']
    hits_set = set(hits_s)
    hits_with_pfam_df = df[df["hit_id"].isin(hits_set)]
    print("number filtered hits: " + str(len(hits_with_pfam_df['hit_id'].unique())))
    return hits_with_pfam_df

In [151]:
query_ids_suffixes = ["Acetyl"]
#get_hits_with_pfam_df(query_ids_suffixes, pfam_df)

In [148]:
pd.set_option('display.max_rows', None)
Mer_hits_df = get_hits_with_pfam_df(["Mer"], pfam_df)
np.mean(Mer_hits_df["target_length"])

number filtered hits: 27


195.35294117647058

### analyze pfams in high icity genes near interesting hits

In [163]:
pd.set_option('display.max_rows', 7)
baitlists_hits = [ast.literal_eval(baitlist) for baitlist in Mer_hits_df['baitp100s'].unique()]

In [167]:
allbaits_hits = []
for baitlist in baitlists_hits:
    allbaits_hits += baitlist
allbaits_hits = set(allbaits_hits)

In [186]:
baitlists_all = [ast.literal_eval(baitlist) for baitlist in pfam_df['baitp100s'].unique()]
baitlists_expanded = []
for baitlist in baitlists_all:
    for bait in baitlist:
        if bait in allbaits_hits:
            baitlists_expanded.append(str(baitlist))
            break

In [189]:
len(baitlists_expanded), len(baitlists_hits)

(47, 27)

In [195]:
def get_icityhits_near_pfam_hits_df(query_ids_suffixes, df):
    all_query_ids = [pfam for pfam in list(df['query_id'].sort_values().unique()) if type(pfam) == str]
    query_ids = []
    for query_ids_suffix in query_ids_suffixes:
        query_ids += [pfam for pfam in all_query_ids if query_ids_suffix in pfam]    
    hits_s = df[df["query_id"].isin(query_ids)]['hit_id']
    hits_set = set(hits_s)
    hits_with_pfam_df = df[df["hit_id"].isin(hits_set)]
    
    baitlists_hits = [ast.literal_eval(baitlist) for baitlist in hits_with_pfam_df['baitp100s'].unique()]
    allbaits_hits = []
    for baitlist in baitlists_hits:
        allbaits_hits += baitlist
    allbaits_hits = set(allbaits_hits)
    
    baitlists_all = [ast.literal_eval(baitlist) for baitlist in df['baitp100s'].unique()]
    baitlists_expanded = []
    for baitlist in baitlists_all:
        for bait in baitlist:
            if bait in allbaits_hits:
                baitlists_expanded.append(str(baitlist))
                break
                
    baitlists_set = set(baitlists_expanded)
    icityhits_near_domainhits_df = df[df["baitp100s"].isin(baitlists_set)].dropna()
    
    print("number pfam filtered hits: " + str(len(hits_with_pfam_df['hit_id'].unique())))
    print("number icity hits near pfam filtered hits: " + str(len(icityhits_near_domainhits_df['hit_id'].unique())))
    return icityhits_near_domainhits_df

In [252]:
# still displays original pfam_hits. also not very informative anymore
get_icityhits_near_pfam_hits_df(["Acetyl"], pfam_df).head()

number pfam filtered hits: 15
number icity hits near pfam filtered hits: 36


,query_id,query_accession,query_len,hit_id,hit_evalue,target_length,baitp100s,icity,numer,denom,is_tnpB
54,AAA_14,PF13173.8,130.0,6d7ff89d8cbf492f5d,1.100000e-10,375.0,"['d02c739d10a0ee4ce9', '6220aa1ee56afc8897', '...",0.818182,9,11,0
55,AAA_16,PF13191.8,170.0,6d7ff89d8cbf492f5d,5.100000e-07,375.0,"['d02c739d10a0ee4ce9', '6220aa1ee56afc8897', '...",0.818182,9,11,0
56,AAA_22,PF13401.8,137.0,6d7ff89d8cbf492f5d,7.400000e-06,375.0,"['d02c739d10a0ee4ce9', '6220aa1ee56afc8897', '...",0.818182,9,11,0
57,ATPase_2,PF01637.20,233.0,6d7ff89d8cbf492f5d,4.200000e-21,375.0,"['d02c739d10a0ee4ce9', '6220aa1ee56afc8897', '...",0.818182,9,11,0
132,Abhydrolase_1,PF00561.22,257.0,146dc63230b0b7aa52,7.800000e-25,310.0,"['3e3b75f90739c0c41e', 'ee5c8bb3c9731c783a', '...",0.722222,26,36,0


### analyze tnpBs without catalytic domains

In [198]:
inactive_tnpBs = []
with open ("../tnpBs/_inactive_tnpBs.faa") as handle:
    for rec in SeqIO.parse(handle, 'fasta'):
        sequence = rec.seq
        pid = rec.id
        inactive_tnpBs.append(pid)

In [203]:
sum([hit_id in inactive_tnpBs_set for hit_id in pfam_df["hit_id"]])

0

In [201]:
inactive_tnpBs_set = set(inactive_tnpBs)

In [225]:
def get_hits_with_inactive_tnpB_bait_df(inactive_tnpBs, df):
#     all_query_ids = [pfam for pfam in list(pfam_df['query_id'].sort_values().unique()) if type(pfam) == str]
#     query_ids = []
#     for query_ids_suffix in query_ids_suffixes:
#         query_ids += [pfam for pfam in all_query_ids if query_ids_suffix in pfam ]    
#     hits_s = df[df["query_id"].isin(query_ids)]['hit_id']
#     hits_set = set(hits_s)
#     hits_with_pfam_df = df[df["hit_id"].isin(hits_set)]
    
    
    baitlists_all = [ast.literal_eval(baitlist) for baitlist in df['baitp100s'].unique()]
    baitlists_expanded = []
    for baitlist in baitlists_all:
        for bait in baitlist:
            if bait in inactive_tnpBs:
                baitlists_expanded.append(str(baitlist))
                break
    hits_with_inactive_tnpB_df = df[df["baitp100s"].isin(baitlists_expanded)].dropna()
    tnpB_hits_coloc_inactive_tnpB =  hits_with_inactive_tnpB_df[hits_with_inactive_tnpB_df['is_tnpB'] == 1]
                
    print("number hits with >1 inactive tnpB bait: " + str(len(hits_with_inactive_tnpB_df['hit_id'].unique())))
    print("number tnpB hits with >1 inactive tnpB bait: " + str(len(tnpB_hits_coloc_inactive_tnpB['hit_id'].unique())))
    return hits_with_inactive_tnpB_df

In [227]:
pd.set_option('display.max_rows', None)
get_hits_with_inactive_tnpB_bait_df(inactive_tnpBs, pfam_df)

number hits with >1 inactive tnpB bait: 27
number tnpB hits with >1 inactive tnpB bait: 1


,query_id,query_accession,query_len,hit_id,hit_evalue,target_length,baitp100s,icity,numer,denom,is_tnpB
15,4HBT,PF03061.24,79.0,a0f27f678af79a25fd,1.500000e-12,169.0,"['d10365dfead3fd2505', 'e646b69bbf42952fac', '...",0.846154,11,13,0
16,DUF4442,PF14539.8,132.0,a0f27f678af79a25fd,1.900000e-05,169.0,"['d10365dfead3fd2505', 'e646b69bbf42952fac', '...",0.846154,11,13,0
35,AAA,PF00004.31,132.0,c7a1b8f01c5b7c48a2,4.800000e-06,1986.0,"['b840bad1d0194681f8', '8be4199a7b3eb07fd5', '...",0.750000,24,32,0
36,AAA_16,PF13191.8,170.0,c7a1b8f01c5b7c48a2,5.000000e-05,1986.0,"['b840bad1d0194681f8', '8be4199a7b3eb07fd5', '...",0.750000,24,32,0
37,AAA_22,PF13401.8,137.0,c7a1b8f01c5b7c48a2,1.100000e-07,1986.0,"['b840bad1d0194681f8', '8be4199a7b3eb07fd5', '...",0.750000,24,32,0
38,ANAPC4_WD40,PF12894.9,92.0,c7a1b8f01c5b7c48a2,9.400000e-45,1986.0,"['b840bad1d0194681f8', '8be4199a7b3eb07fd5', '...",0.750000,24,32,0
39,Ge1_WD40,PF16529.7,328.0,c7a1b8f01c5b7c48a2,1.800000e-24,1986.0,"['b840bad1d0194681f8', '8be4199a7b3eb07fd5', '...",0.750000,24,32,0
40,NACHT,PF05729.14,166.0,c7a1b8f01c5b7c48a2,3.600000e-11,1986.0,"['b840bad1d0194681f8', '8be4199a7b3eb07fd5', '...",0.750000,24,32,0
41,Nbas_N,PF15492.8,282.0,c7a1b8f01c5b7c48a2,5.900000e-05,1986.0,"['b840bad1d0194681f8', '8be4199a7b3eb07fd5', '...",0.750000,24,32,0
42,Nup160,PF11715.10,547.0,c7a1b8f01c5b7c48a2,3.900000e-06,1986.0,"['b840bad1d0194681f8', '8be4199a7b3eb07fd5', '...",0.750000,24,32,0


In [206]:
inactive_tnpBs

['2954508ddf88f78a5e',
 '73d5185a85348cf7b5',
 'fc32217464f8bf8376',
 '0da5a5d756496ec587',
 '15ce911abbf167916e',
 '70e9d1aa2ac3c57a62',
 '476fa0561841de3b03',
 '0648051c710ffc3f99',
 'b9cc7b8a78f75d3bc5',
 '0f08a20cfe029876e3',
 'bbfd4d4a48a938d8b8',
 'd282cda4fcf2772f8b',
 '0639750e6f677cd5b0',
 'dd214aaeb8a31e8fe2',
 '349bfea99dfac199dc',
 '300dec149b54de75dc',
 '93fcfb582d1575f9eb',
 '744e34a3f75a21ecb8',
 '3cf7abaaf6fddcaab4',
 '19800dc11e240a4807',
 '6a285900ee566adb79',
 '702076d18721ab4b19',
 '252302b5d0d84f1bdc',
 'ee5c498bf075a0ad33',
 '7fbf3dca298592c1c7']